# Корреляция Пирсона и др

In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на маллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.

Есть ли связь между жёсткостью воды и средней годовой смертностью? Посчитайте значение коэффициента корреляции Пирсона между этими признаками, округлите его до четырёх знаков после десятичной точки.

In [2]:
water = pd.read_table('water.txt')
water.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [3]:
stats.pearsonr(water.hardness, water.mortality)

(-0.6548486232042465, 1.0331339260349571e-08)

В предыдущей задаче посчитайте значение коэффициента корреляции Спирмена между средней годовой смертностью и жёсткостью воды. Округлите до четырёх знаков после десятичной точки.


In [4]:
np.round(stats.spearmanr(water.hardness, water.mortality)[0], 4)

-0.6317

In [5]:
np.round(stats.pearsonr(water.hardness, water.mortality)[0], 4)

-0.6548

In [6]:
np.round(stats.pearsonr(water[water.location == 'South'].hardness,
                        water[water.location == 'South'].mortality)[0], 4)

-0.6022

In [7]:
np.round(stats.pearsonr(water[water.location == 'North'].hardness,
                        water[water.location == 'North'].mortality)[0], 4)

-0.3686

Среди респондентов General Social Survey 2014 года хотя бы раз в месяц проводят вечер в баре 203 женщины и 239 мужчин; реже, чем раз в месяц, это делают 718 женщин и 515 мужчин.

Посчитайте значение коэффициента корреляции Мэтьюса между полом и частотой похода в бары. Округлите значение до трёх знаков после десятичной точки.


In [8]:
def Mattews(a, b, c, d):
    u = a * d - b * c
    t = np.sqrt( (a + b ) * (a + c) * (b + d) * (c + d) )
    return u * 1.0 / t

In [9]:
np.round(Mattews( 239, 515, 203, 718), 4)

0.109

In [10]:
X = np.array([[239, 515], [203, 718]])

В предыдущей задаче проверьте, значимо ли коэффициент корреляции Мэтьюса отличается от нуля. Посчитайте достигаемый уровень значимости; используйте функцию scipy.stats.chi2_contingency

In [11]:
stats.chi2_contingency(X)

(19.40753078854304,
 1.0558987006638725e-05,
 1,
 array([[198.96597015, 555.03402985],
        [243.03402985, 677.96597015]]))

In [12]:
def Mattews2(X):
    a = X[0,0]
    b = X[0,1]
    c = X[1,0]
    d = X[1,1]
    u = a * d - b * c
    t = np.sqrt( (a + b ) * (a + c) * (b + d) * (c + d) )
    return u * 1.0 / t

В предыдущей задаче давайте попробуем ответить на немного другой вопрос: отличаются ли доля мужчин и доля женщин, относительно часто проводящих вечера в баре? Постройте 95% доверительный интервал для разности долей, вычитая долю женщин из доли мужчин. Чему равна его нижняя граница? Округлите до четырёх знаков после десятичной точки.



Проверьте гипотезу о равенстве долей любителей часто проводить вечера в баре среди мужчин и женщин. Посчитайте достигаемый уровень значимости, используя двустороннюю альтернативу. 

In [13]:
def proportions_diff_z_stat_rel(f, g, n):
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [14]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [15]:
def proportions_diff_confint_ind(p1, p2, n1, n2, alpha = 0.05):    
    z = stats.norm.ppf(1 - alpha / 2.)
    
#     p1 = float(sum(sample1)) / len(sample1)
#     p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ n1  + p2 * (1 - p2)/ n2)
    
    return (left_boundary, right_boundary)

In [16]:
print "95%% confidence interval for a difference between proportions: [%f, %f]" %\
      proportions_diff_confint_ind(239./(239+515), 203./(203+718) , 239+515, 203+718)

95% confidence interval for a difference between proportions: [0.053905, 0.139222]


In [17]:
import scipy

In [18]:
def proportions_diff_z_stat_ind(p1, p2, n1, n2):
#     n1 = len(sample1)
#     n2 = len(sample2)
    
#     p1 = float(sum(sample1)) / n1
#     p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [19]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [20]:
print "p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_ind(239./(239+515), 203./(203+718) , 239+515, 203+718))

p-value: 0.000008


Посмотрим на данные General Social Survey 2014 года и проанализируем, как связаны ответы на вопросы "Счастливы ли вы?" и "Довольны ли вы вашим финансовым положением?"

| Не доволен | Более или менее |	Доволен| 

Не очень счастлив	| 197| 	111	| 33| 

Достаточно счастлив	| 382| 	685	| 331| 

Очень счастлив	| 110	| 342| 	333| 

Чему равно значение статистики хи-квадрат для этой таблицы сопряжённости? Округлите ответ до четырёх знаков после десятичной точки.

In [21]:
Happy = np.array([[197, 111, 33], [382, 685, 331], [110, 342, 333]])

In [22]:
stats.chi2_contingency(Happy)

(293.68311039689746,
 2.4964299580093467e-62,
 4,
 array([[ 93.08597464, 153.74722662,  94.16679873],
        [381.6251981 , 630.318542  , 386.0562599 ],
        [214.28882726, 353.93423138, 216.77694136]]))

На данных из предыдущего вопроса посчитайте значение достигаемого уровня значимости. 

In [23]:
def Kramer(hi, k1, k2, n):
    return np.sqrt(hi / (n * (min(k1, k2) - 1)))

In [24]:
Kramer(stats.chi2_contingency(Happy)[0], 3, 3, Happy.sum())

0.2412013934500338